# TimeSeries DataAnalytics Tutorial



In [1]:
import numpy as np
import chart_studio.plotly as py
import plotly.express as px
import plotly.tools as tls
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import requests
import json
#import matplotlib.pyplot as plt

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
sc = pyspark.SparkContext(appName="TimeSeries")

## I/ Simple IRIS example to check PySpark

In [ ]:
import os
import pandas as pd
data_dir = '/work/irlin355_1/gratienj/BigData/DataLakeBenchProject/python/TimeSeries/DataAnalytics'
filename = os.path.join(data_dir,'iris.csv')
df = pd.read_csv(filename)

In [ ]:
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
sqlContext = SQLContext(sc)
schema = StructType([StructField("sepal_length", DoubleType(), True),
                     StructField("sepal_width",  DoubleType(), True),
                     StructField("petal_length", DoubleType(), True),
                     StructField("petal_width",  DoubleType(), True),
                     StructField("variety",      StringType(), True),
                    ])
spark_df = sqlContext.createDataFrame(df,schema=schema)

In [ ]:
spark_df.printSchema()

In [ ]:
spark_df.show()

## I/ SmartGrid Example from File

### A/ Standard method with Pandas

In [106]:
def str_to_timestamp(date):
    dt = datetime.strptime(date, '%d/%m/%Y %H:%M:%S')
    d0 = datetime(2019,1,1,0,0,0,0)
    return int((dt - d0).total_seconds())

In [107]:
def compute_curve(df,day_id,tagname):
    day_df = df.loc[(df['day_id'] == day_id) & (df['tagname'] == tagname )]
    vh_df = day_df[['hour_id','value']].groupby('hour_id').mean().reset_index().sort_values(by='hour_id')
    x = vh_df['hour_id']
    y = vh_df['value']
    return x,y

In [108]:
import os
data_dir = '/work/irlin355_1/gratienj/BigData/DigitalSandBox/Data/TimeSeries/SmartGridData/Cryolite/20190101'
filename = os.path.join(data_dir,'OneMonth.csv')
df = pd.read_csv(filename, sep=';')
#tagnames = ['CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow']
tagnames = df.tagname.unique()

In [109]:
%%time
df['value'] = pd.to_numeric(df['value'])
df['quality'] = pd.to_numeric(df['value'])
df['timestamp'] = df['timestamp'].apply(str_to_timestamp)
df['day_id'] = df['timestamp']//(3600*24)
df['hour_id'] = df['timestamp'] % (3600*24) // 3600
results =[]
for tag_id,tagname in enumerate(tagnames):
    for day_id in range(30):
        results.append(compute_curve(df,day_id,tagname))

CPU times: user 17.4 s, sys: 0 ns, total: 17.4 s
Wall time: 17.6 s


### B/ PySpark method

In [5]:
import os
data_dir = '/work/irlin355_1/gratienj/BigData/DigitalSandBox/Data/TimeSeries/SmartGridData/Cryolite/20190101'
filename = os.path.join(data_dir,'OneMonth.csv')
df = pd.read_csv(filename, sep=';')
df['value'] = pd.to_numeric(df['value'])
df['quality'] = pd.to_numeric(df['value'])

In [8]:
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
sqlContext = SQLContext(sc)

In [9]:
Schema = StructType([ StructField("timestamp", StringType(),    True),
                      StructField("tagname",   StringType(),  True),
                      StructField("value",     DoubleType(),  True),
                      StructField("quality",   DoubleType(),  True)
                    ])

In [10]:
spark_df = sqlContext.createDataFrame(df,schema=Schema)

In [11]:
spark_df.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- tagname: string (nullable = true)
 |-- value: double (nullable = true)
 |-- quality: double (nullable = true)



In [12]:
spark_df.show()

+-------------------+--------------------+-----+-------+
|          timestamp|             tagname|value|quality|
+-------------------+--------------------+-----+-------+
|01/01/2019 09:15:12|CRY.CENTRALE_SOLA...|  1.0|    1.0|
|01/01/2019 09:15:18|CRY.CENTRALE_SOLA...|  0.0|    0.0|
|01/01/2019 09:15:37|CRY.CENTRALE_SOLA...|  1.0|    1.0|
|01/01/2019 09:15:43|CRY.CENTRALE_SOLA...|  0.0|    0.0|
|01/01/2019 09:15:53|CRY.CENTRALE_SOLA...|  1.0|    1.0|
|01/01/2019 09:15:58|CRY.CENTRALE_SOLA...|  0.0|    0.0|
|01/01/2019 09:16:13|CRY.CENTRALE_SOLA...|  1.0|    1.0|
|01/01/2019 09:16:23|CRY.CENTRALE_SOLA...|  0.0|    0.0|
|01/01/2019 09:16:53|CRY.CENTRALE_SOLA...|  1.0|    1.0|
|01/01/2019 09:16:59|CRY.CENTRALE_SOLA...|  0.0|    0.0|
|01/01/2019 09:17:03|CRY.CENTRALE_SOLA...|  1.0|    1.0|
|01/01/2019 09:17:09|CRY.CENTRALE_SOLA...|  0.0|    0.0|
|01/01/2019 09:17:18|CRY.CENTRALE_SOLA...|  1.0|    1.0|
|01/01/2019 09:17:24|CRY.CENTRALE_SOLA...|  0.0|    0.0|
|01/01/2019 09:17:29|CRY.CENTRA

In [13]:
def str_to_timestamp(date):
    dt = datetime.strptime(date, '%d/%m/%Y %H:%M:%S')
    d0 = datetime(2019,1,1,0,0,0,0)
    return int((dt - d0).total_seconds())

In [14]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, LongType
F1 = udf(lambda x: str_to_timestamp(x), LongType())
dayId = udf(lambda x : x//(3600*24))
hourId = udf(lambda x : x % (3600*24) // 3600)

In [23]:
def compute_curve(tag_df,day_id):
    day_df = tag_df.filter(tag_df.day_id == day_id)
    rdd = day_df.select('hour_id','value').groupby('hour_id').mean()
    x = [x["hour_id"] for x in rdd.select("hour_id").collect()]
    y = [x["avg(value)"] for x in rdd.select("avg(value)").collect()]
    return sorted(zip(x,y))

In [29]:
%%time
tagnames = df.tagname.unique()
spark_df2 = spark_df.withColumn("timestamp_sec",  F1(spark_df["timestamp"]))
spark_df3 = spark_df2.withColumn("day_id",dayId(spark_df2['timestamp_sec']))
spark_df4 = spark_df3.withColumn("hour_id",hourId(spark_df3['timestamp_sec']))
results =[]
for tag_id,tagname in enumerate(tagnames):
    tag_df = spark_df4.filter(spark_df4.tagname == tagname)
    for day_id in range(30):
        results.append(compute_curve(tag_df,day_id))

CPU times: user 4.75 s, sys: 1.81 s, total: 6.56 s
Wall time: 10min 20s


### C/ Data from DataBase

In [30]:
import sys
sys.path.append("../TimeSeriesTools")
import mongodb_utils
db_host= 'localhost'
port = '27018'
db_name='TimeSeriesBench'
mongodb_client = mongodb_utils.mongodb_connect(db_host, port)

Trying to connect to MongoDB server: localhost on port: 27018


In [31]:
def get_collection_scheme(db_name,scheme_name):
    db = mongodb_client[db_name]
    schemes_coll = db['schemes']
    scheme = schemes_coll.find({"name":scheme_name})
    return scheme

In [32]:
def mongodb_find_all_data(db_name,coll_name,scheme):
    data = mongodb_utils.get_all_data(mongodb_client,db_name,coll_name,scheme)
    return data

In [33]:
scheme = get_collection_scheme(db_name,'SmartGrid')

In [96]:
%%time
coll_name='SmartGridCryolite20190101OneMonthBS10000d'
data = mongodb_find_all_data(db_name,coll_name,scheme[0]['value'])
print("number of docs",len(data))

769598  documents found
number of docs 769598
CPU times: user 4.5 s, sys: 7.99 ms, total: 4.51 s
Wall time: 4.81 s


In [97]:
for i,d in enumerate(data):
    d['_id'] = i

In [98]:
%%time
df = pd.DataFrame(data)
df[0:5]

CPU times: user 1.05 s, sys: 0 ns, total: 1.05 s
Wall time: 1.05 s


,_id,timestamp,tagname,value,quality
0,0,01/01/2019 09:15:12,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0
1,1,01/01/2019 09:15:18,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,0.000000000,100.0
2,2,01/01/2019 09:15:37,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0
3,3,01/01/2019 09:15:43,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,0.000000000,100.0
4,4,01/01/2019 09:15:53,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0


In [99]:
Schema = StructType([ StructField("_id",       LongType(),  True),
                      StructField("timestamp", StringType(),    True),
                      StructField("tagname",   StringType(),  True),
                      StructField("value",     StringType(),  True),
                      StructField("quality",   StringType(),  True)
                    ])

In [100]:
spark_df = sqlContext.createDataFrame(df,schema=Schema)

In [102]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="tagname", outputCol="tagindex")
indexed_df = indexer.fit(spark_df).transform(spark_df)

In [103]:
tagnames = df.tagname.unique()
max_tag_index = len(tagnames)+1
dayId = udf(lambda x : x//(3600*24))
hourId = udf(lambda x,y : int(y) * 24 + (x % (3600*24) // 3600 )  )

In [105]:
%%time
spark_df1 = indexed_df.withColumn("timestamp_sec",  F1(indexed_df["timestamp"]))
spark_df2 = spark_df1.withColumn("value_d",  spark_df1["value"].cast("double"))
spark_df3 = spark_df2.withColumn("day_id",dayId(spark_df2['timestamp_sec']))
spark_df4 = spark_df3.withColumn("hour_id",hourId(spark_df3['timestamp_sec'],spark_df3['tagindex']))
rdd = spark_df4.select('hour_id','value_d').groupby('hour_id').mean()
x = [x["hour_id"] for x in rdd.select("hour_id").collect()]
y = [x["avg(value_d)"] for x in rdd.select("avg(value_d)").collect()]
result = sorted(zip(x,y))

CPU times: user 42.5 ms, sys: 0 ns, total: 42.5 ms
Wall time: 8.55 s
